<a href="https://colab.research.google.com/github/XHLin-gamer/code_acrostic/blob/main/CC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Preparation, it may takes some times
import os
import subprocess
from time import time
tic = time()

%store -r

root_dir = "/content"
repo_dir = "/content/MarkLLM"

def clone_repo():
    repo_url = "https://github.com/THU-BPM/MarkLLM"
    subprocess.run(["git", "clone", repo_url, repo_dir], check=True)

def install_dependencies():
    subprocess.run(["pip", "install", "-r", "requirements.txt"], check=True)

os.chdir(root_dir)
clone_repo()
os.chdir(repo_dir)
install_dependencies()


toc = time() - tic
print(f"Preparation Finished with {toc} seconds")

Preparation Finished with 89.35433459281921 seconds


In [2]:
import gc
import torch
import os
import json
from watermark.auto_watermark import AutoWatermark
from utils.transformers_config import TransformersConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# Clean gpu memory
if torch.cuda.is_available():
  gc.collect()
  torch.cuda.empty_cache()
  with torch.no_grad():
      torch.cuda.empty_cache()

# download the model's weight from hugging-face
Model_repo = "deepseek-ai/deepseek-coder-1.3b-instruct"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(Model_repo, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(Model_repo, trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [3]:
transformers_config = TransformersConfig(model=model.to(device),
                      tokenizer=tokenizer,
                      vocab_size=32256,
                      device=device,
                      max_new_tokens=512)

In [4]:
!wget -O /content/HumanEval.jsonl.gz https://raw.githubusercontent.com/openai/human-eval/master/data/HumanEval.jsonl.gz

--2024-08-26 11:39:45--  https://raw.githubusercontent.com/openai/human-eval/master/data/HumanEval.jsonl.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44877 (44K) [application/octet-stream]
Saving to: ‘/content/HumanEval.jsonl.gz’

/content/HumanEval. 100%[===================>]  43.83K  --.-KB/s    in 0.01s   

2024-08-26 11:39:45 (3.86 MB/s) - ‘/content/HumanEval.jsonl.gz’ saved [44877/44877]



In [5]:
!wget -O /content/HumanEval.jsonl.gz https://raw.githubusercontent.com/openai/human-eval/master/data/HumanEval.jsonl.gz
!gunzip /content/HumanEval.jsonl.gz

--2024-08-26 11:39:45--  https://raw.githubusercontent.com/openai/human-eval/master/data/HumanEval.jsonl.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44877 (44K) [application/octet-stream]
Saving to: ‘/content/HumanEval.jsonl.gz’

/content/HumanEval. 100%[===================>]  43.83K  --.-KB/s    in 0.01s   

2024-08-26 11:39:45 (3.59 MB/s) - ‘/content/HumanEval.jsonl.gz’ saved [44877/44877]



In [6]:
from datasets import load_dataset

ds = load_dataset("h4iku/coconut_python2010")

Generating train split:   0%|          | 0/480777 [00:00<?, ? examples/s]

In [43]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
# scripts_token = [tokenizer.encode(script) for script in ds['train']['context']]
# 16384
from tqdm.notebook import tqdm
corpus = {}
for script in tqdm(ds['train']['context']):
    script = tokenizer.encode(script[16384:])
    for pre, after in zip(script[:-1], script[1:]):
        if pre == after: continue
        if pre not in corpus: corpus[pre] = {}
        if after not in corpus[pre]: corpus[pre][after] = 0
        corpus[pre][after] += 1
corpus_statistic = {}
import torch
from torch.distributions import Categorical

for pre in corpus:
    # print(list(corpus[pre].values()))
    ids = torch.tensor(list(corpus[pre].values()))
    # variance = statistics.pvariance(ids)
    ids = torch.clip(ids, max=2e8)
    # soft_ids = torch.softmax(ids, dim=0)
    soft_ids = ids / torch.sum(ids)
    # entropy_ids = -np.sum(soft_ids*np.log2(soft_ids))
    # print()

    corpus_statistic.update(
        # {pre: variance / sum(ids)}
        # {pre: torch.max(soft_ids)}
        # {pre: soft_ids.}
        {pre:Categorical(probs = soft_ids).entropy()}
    )
    # break
white_list = {}
for c in corpus_statistic:
    if corpus_statistic[c] > 2.7:
        # print(c, corpus_statistic[c])
        white_list.update(
            {c: corpus_statistic[c]}
        )
len(white_list) / len(corpus_statistic)
import json
with open('./white_list4.json', 'w') as f:
    param = {
        'white_list': list(white_list.keys())
    }
    json.dump(param, f)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/480777 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (32624 > 16384). Running this sequence through the model will result in indexing errors


In [7]:
import re
# raw output accumulation
import pandas
from tqdm.notebook import tqdm
df = pandas.read_json('/content/HumanEval.jsonl', orient='records', lines=True)


def extract_function_definitions(script_content):
    functions = []
    lines = script_content.splitlines()
    inside_function = False
    function_name = ""
    function_code = []

    for line in lines:
        stripped_line = line.strip()

        # Detect the start of a function definition
        if stripped_line.startswith("def ") and not inside_function:
            inside_function = True
            function_name = stripped_line.split("(")[0][4:]  # Extract function name
            function_code.append(line)

        # Continue collecting lines of the function
        elif inside_function:
            # If the line is indented, it's part of the current function
            if line.startswith(" ") or line.startswith("\t"):
                function_code.append(line)
            # If the line isn't indented, the function definition has ended
            else:
                functions.append((function_name, "\n".join(function_code)))
                inside_function = False
                function_code = []

        # If the end of the script is reached and still inside a function
    if inside_function:
        functions.append((function_name, "\n".join(function_code)))

    return function_name


def extract_triple_quotes_content(text):
    # Regular expression pattern to find content between triple quotes
    pattern = r'"""(.*?)"""'

    # Find all matches in the text
    matches = re.findall(pattern, text, re.DOTALL)
    if not matches:
        pattern = r"'''(.*?)'''"
        matches = re.findall(pattern, text, re.DOTALL)
    return matches

def split_and_keep_delimiter(text, delimiter):
    # Regular expression pattern to split and keep the delimiter
    pattern = f'({re.escape(delimiter)})'

    # Split the text based on the pattern
    parts = re.split(pattern, text)

    return parts


def promptify(prompt):
    func_name = extract_function_definitions(prompt)
    content: str = extract_triple_quotes_content(prompt)[0]
    # content: str = extract_triple_quotes_content(df['prompt'][30])[0]
    # c = content[0].split(">>>")
    content = split_and_keep_delimiter(content, '>>>')
    if len(content) == 1:
        content = content[0].split('Examples:')
    if len(content) == 1:
        content = content[0].split('Examples')
    if len(content) == 1:
        content = content[0].split('Example:')
    if len(content) == 1:
        content = content[0].split('Example')
    # print(" ".join(content[0].replace('\n', ' ').split()))
    # content =
    problem = " ".join(content[0].replace('\n', ' ').split())
    test = "".join(content[1:])
    PROMPT = f"""Problem: {problem}

Tests:
    {test}

Implementation:
```python
def {func_name}"""
    return PROMPT
print(promptify(df['prompt'][0]))

Problem: Check if in given list of numbers, are any two numbers closer to each other than given threshold.

Tests:
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    

Implementation:
```python
def has_close_elements


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def generate_raw_context_without_watermark():
    try:
      os.mkdir('/content/raw_ds_prompted_1.3b')
    except:
      print('/content/raw_ds_prompted_1.3b already exist')
    for idx, prompt in enumerate(tqdm(df['prompt'])):
        with open(f'/content/raw_ds_prompted_1.3b/{idx}.py','w') as f:
            input_text = promptify(prompt)
            inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
            # inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
            outputs = model.generate(inputs, max_length=512, do_sample = False)
            script = tokenizer.decode(outputs[0], skip_special_tokens=True)
            f.write(script)
            f.close()
generate_raw_context_without_watermark()

  0%|          | 0/164 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask 

In [10]:
import json

import torch

from utils.utils import load_config_file
from MarkLLM.utils.transformers_config import TransformersConfig
from MarkLLM.watermark.kgw.kgw import KGWConfig, KGWUtils, KGWLogitsProcessor, KGW


class CC_Config(KGWConfig):
    def __init__(self, algorithm_config: str, transformers_config: TransformersConfig, white_list_path: str, *args, **kwargs) -> None:
        """
            Initialize the CC configuration.

            Parameters:
                algorithm_config (str): Path to the algorithm configuration file.
                transformers_config (TransformersConfig): Configuration for the transformers model.
        """
        if algorithm_config is None:
            config_dict = load_config_file('config/CC.json')
        else:
            config_dict = load_config_file(algorithm_config)
        if config_dict['algorithm_name'] != 'CC':
            raise AlgorithmNameMismatchError('CC', config_dict['algorithm_name'])

        self.gamma = config_dict['gamma']
        print(f"Gamma: {self.gamma}")
        self.delta = config_dict['delta']
        self.hash_key = config_dict['hash_key']
        self.z_threshold = config_dict['z_threshold']
        self.prefix_length = config_dict['prefix_length']
        self.f_scheme = config_dict['f_scheme']
        self.window_scheme = config_dict['window_scheme']

        self.generation_model = transformers_config.model
        self.generation_tokenizer = transformers_config.tokenizer
        self.vocab_size = transformers_config.vocab_size
        self.device = transformers_config.device
        self.gen_kwargs = transformers_config.gen_kwargs

        self.white_list = []
        with open(white_list_path, 'r') as f:
            self.white_list = json.load(f)['white_list']
            f.close()


class CC_Utils(KGWUtils):
    def __init__(self, config: CC_Config, *args, **kwargs) -> None:
        super().__init__(config, *args, **kwargs)

    def score_sequence(self, input_ids: torch.Tensor) -> tuple[float, list[int]]:
        """Overload The original score approach,
           and only score the token after the white list"""
        num_tokens_scored = len(input_ids) - self.config.prefix_length
        if num_tokens_scored < 1:
            raise ValueError(
                (
                    f"Must have at least {1} token to score after "
                    f"the first min_prefix_len={self.config.prefix_length} tokens required by the seeding scheme."
                )
            )
        num_tokens_scored = 0

        green_token_count = 0
        green_token_flags = [-1 for _ in range(self.config.prefix_length)]

        for idx in range(self.config.prefix_length, len(input_ids)):
            curr_token = input_ids[idx]
            pre_token = input_ids[idx - 1]
            # skip the token which not in the white list
            if pre_token not in self.config.white_list: continue

            num_tokens_scored += 1
            greenlist_ids = self.get_greenlist_ids(input_ids[:idx])

            if curr_token in greenlist_ids:
                green_token_count += 1
                green_token_flags.append(1)
            else:
                green_token_flags.append(0)
        # print(green_token_count, num_tokens_scored)
        z_score = self._compute_z_score(green_token_count, num_tokens_scored)
        return z_score, green_token_flags


class CC_Processor(KGWLogitsProcessor):
    def __init__(self, config: CC_Config, utils: CC_Utils, *args, **kwargs) -> None:
        super().__init__(config, utils, *args, **kwargs)
        self.white_list = self.config.white_list

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        # return super().__call__(input_ids, scores)
        if input_ids.shape[-1] < self.config.prefix_length:
            return scores

        batched_greenlist_ids = [None for _ in range(input_ids.shape[0])]

        for b_idx in range(input_ids.shape[0]):
            greenlist_ids = self.utils.get_greenlist_ids(input_ids[b_idx])
            # only the tokens in the white list will injected with watermark
            if int(input_ids[b_idx][-1]) in self.white_list:
              batched_greenlist_ids[b_idx] = greenlist_ids

        green_tokens_mask = self._calc_greenlist_mask(scores=scores, greenlist_token_ids=batched_greenlist_ids)

        scores = self._bias_greenlist_logits(scores=scores, greenlist_mask=green_tokens_mask, greenlist_bias=self.config.delta )
        return scores

class CC(KGW):
    def __init__(self, algorithm_config: str, transformers_config: TransformersConfig, white_list_path: str,  *args, **kwargs) -> None:
        # super().__init__(algorithm_config, transformers_config, *args, **kwargs)
        self.config = CC_Config(algorithm_config, transformers_config, white_list_path)
        self.utils = CC_Utils(self.config)
        self.logits_processor = CC_Processor(self.config, self.utils)


In [11]:
myWatermark = CC('/content/CC.json', transformers_config,'/content/white_list4.json' )

Gamma: 0.1


In [ ]:
prompt = """Problem: Input to this function is a string containing multiple groups of nested parentheses. Your goal is to separate those group into separate strings and return the list of those. Separate groups are balanced (each open brace is properly closed) and not nested within each other Ignore any spaces in the input string.

Tests:
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']


Implementation:
```python
def separate_paren_groups"""
watermarked_text = myWatermark.generate_watermarked_text(prompt)
print(watermarked_text)
unwatermarked_text = myWatermark.generate_unwatermarked_text(prompt)
print('------------')
print(unwatermarked_text)

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Problem: Input to this function is a string containing multiple groups of nested parentheses. Your goal is to separate those group into separate strings and return the list of those. Separate groups are balanced (each open brace is properly closed) and not nested within each other Ignore any spaces in the input string.

Tests:
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    

Implementation:
```python
def separate_paren_groups(input_string):
    result = []
    stack = []
    temp = ''
    for char in input_string:
        if char == ' ':
            continue
        if char == '(':
            stack.append(char)
            temp += char
        if char == ')':
            stack.append(char)
            temp += char
            if stack[-1] == '(':
                result.append(temp)
                temp = ''
            else:
                temp = temp[:-1] + temp[-1]
    return result
```

This function works by iterating over each character in t

In [ ]:
detect_result1 = myWatermark.detect_watermark(watermarked_text)
detect_result2 = myWatermark.detect_watermark(unwatermarked_text)
detect_result1, detect_result2

({'is_watermarked': True, 'score': 6.21191856861985},
 {'is_watermarked': False, 'score': -0.8451542547285166})

In [ ]:
def generate_with_watermark(watermark, folder_name):
  try:
    os.mkdir(f'/content/{folder_name}')
  except:
    print(f"{folder_name} already exist")
  for idx, prompt in enumerate(tqdm(df['prompt'])):
      with open(f'/content/{folder_name}/{idx}.py','w') as f:
          prompt = promptify(prompt)
          watermarked_text = watermark.generate_watermarked_text(prompt)
          f.write(watermarked_text)
          f.close()

generate_with_watermark(myWatermark, 'cc')

In [34]:
os.chdir('/content/MarkLLM')
algorithm_name = 'SWEET'
SWEET = AutoWatermark.load(f'{algorithm_name}', algorithm_config=f'./config/{algorithm_name}.json', transformers_config=transformers_config)
algorithm_name = 'KGW'
KGW = AutoWatermark.load(f'{algorithm_name}', algorithm_config=f'./config/{algorithm_name}.json', transformers_config=transformers_config)

In [ ]:
generate_with_watermark(SWEET, 'sweet')

sweet already exist


  0%|          | 0/164 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end gene

In [ ]:
generate_with_watermark(KGW, 'kgw')
# 89

  0%|          | 0/164 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end gene

In [12]:
import re
def extract_triple_quotes_content(text):
    # Regular expression pattern to find content between triple quotes
    pattern = r'```python(.*?)```'

    # Find all matches in the text
    matches = re.findall(pattern, text, re.DOTALL)
    return matches

In [13]:
def detect_watermark(watermark, folder_name):
  is_mark = list()
  score = list()
  for idx, prompt in enumerate(tqdm(df['prompt'])):
      with open(f'/content/{folder_name}/{idx}.py','r') as f:
          watermarked_text = f.read()
          t = extract_triple_quotes_content(watermarked_text)
          if len(t) > 0:
            # print('hit')
            """temp ban to get non-comment result"""
            watermarked_text = t[0]
          detect_result1 = watermark.detect_watermark(text = watermarked_text)
          # print(detect_result1)
          is_mark.append(detect_result1['is_watermarked'])
          score.append(detect_result1['score'])
          f.close()
  return is_mark, score


In [37]:
is_mark, scores = detect_watermark(KGW, 'kgw')
kgw_score = scores
print(f"TPR for kgw is :{sum(is_mark)/ len(is_mark)}")
is_mark, scores = detect_watermark(KGW, 'raw_ds_prompted_1.3b')
F_kgw_score = scores
print(f"FPR for kgw is :{sum(is_mark)/ len(is_mark)}")
print('--------------')
is_mark, scores = detect_watermark(SWEET, 'sweet')
sweet_score = scores
print(f"TPR for SWEET is :{sum(is_mark)/ len(is_mark)}")
is_mark, scores = detect_watermark(SWEET, 'raw_ds_prompted_1.3b')
F_sweet_score = scores
print(f"FPR for SWEET is :{sum(is_mark)/ len(is_mark)}")
print('--------------')
is_mark, scores = detect_watermark(myWatermark, 'cc')
cc_score = scores
print(f"TPR for CC is :{sum(is_mark)/ len(is_mark)}")
is_mark, scores = detect_watermark(myWatermark, 'raw_ds_prompted_1.3b')
F_cc_score = scores
print(f"FPR for CC is :{sum(is_mark)/ len(is_mark)}")
print('--------------')

  0%|          | 0/164 [00:00<?, ?it/s]

TPR for kgw is :0.17073170731707318


  0%|          | 0/164 [00:00<?, ?it/s]

FPR for kgw is :0.04878048780487805
--------------


  0%|          | 0/164 [00:00<?, ?it/s]

...

FPR for SWEET is :0.018292682926829267
--------------


  0%|          | 0/164 [00:00<?, ?it/s]

TPR for CC is :0.3475609756097561


  0%|          | 0/164 [00:00<?, ?it/s]

FPR for CC is :0.042682926829268296
--------------


In [39]:
def detect_watermark_mix(watermark, folder_name):
  is_mark = list()
  score = list()
  for idx, prompt in enumerate(tqdm(df['prompt'])):
      with open(f'/content/{folder_name}/{idx}.py','r') as f:
          watermarked_text = f.read()
          # t = extract_triple_quotes_content(watermarked_text)
          # if len(t) > 0:
          #   # print('hit')
          #   """temp ban to get non-comment result"""
          #   watermarked_text = t[0]
          detect_result1 = watermark.detect_watermark(text = watermarked_text)
          # print(detect_result1)
          is_mark.append(detect_result1['is_watermarked'])
          score.append(detect_result1['score'])
          f.close()
  return is_mark, score


In [40]:
is_mark, scores = detect_watermark_mix(KGW, 'kgw')
kgw_score = scores
print(f"TPR for kgw mix is :{sum(is_mark)/ len(is_mark)}")
is_mark, scores = detect_watermark_mix(KGW, 'raw_ds_prompted_1.3b')
F_kgw_score = scores
print(f"FPR for kgw mix is :{sum(is_mark)/ len(is_mark)}")
print('--------------')
is_mark, scores = detect_watermark_mix(SWEET, 'sweet')
sweet_score = scores
print(f"TPR for SWEET mix is :{sum(is_mark)/ len(is_mark)}")
is_mark, scores = detect_watermark_mix(SWEET, 'raw_ds_prompted_1.3b')
F_sweet_score = scores
print(f"FPR for SWEET mix is :{sum(is_mark)/ len(is_mark)}")
print('--------------')
is_mark, scores = detect_watermark_mix(myWatermark, 'cc')
cc_score = scores
print(f"TPR for CC mix is :{sum(is_mark)/ len(is_mark)}")
is_mark, scores = detect_watermark_mix(myWatermark, 'raw_ds_prompted_1.3b')
F_cc_score = scores
print(f"FPR for CC mix is :{sum(is_mark)/ len(is_mark)}")
print('--------------')

  0%|          | 0/164 [00:00<?, ?it/s]

TPR for kgw mix is :0.24390243902439024


  0%|          | 0/164 [00:00<?, ?it/s]

FPR for kgw mix is :0.018292682926829267
--------------


  0%|          | 0/164 [00:00<?, ?it/s]

...

FPR for SWEET mix is :0.042682926829268296
--------------


  0%|          | 0/164 [00:00<?, ?it/s]

TPR for CC mix is :0.8536585365853658


  0%|          | 0/164 [00:00<?, ?it/s]

FPR for CC mix is :0.1402439024390244
--------------


In [31]:
from sklearn.metrics import roc_auc_score
def calc_auroc(true_scores, false_scores):
    # Example: true labels and predicted probabilities
    y_scores = [*true_scores, *false_scores]  # True binary labels
    y_true = [*[1 for _ in true_scores], *[0 for _ in false_scores]]  # Predicted probabilities  or scores

    # print(y_true)
    # # Compute AUROC
    auroc = roc_auc_score(y_true, y_scores)
    return auroc

print(f"AUROC:")
print(f"KGW: {calc_auroc(kgw_score, F_kgw_score)}")
print(f"SWEET: {calc_auroc(sweet_score, F_sweet_score)}")
print(f"CC: {calc_auroc(cc_score, F_cc_score)}")

AUROC:
KGW: 0.684506989886972
SWEET: 0.6924821534800714
CC: 0.7721222486615111


In [19]:
os.chdir('/content')
!git clone https://github.com/openai/human-eval.git /content/human_eval
!pip install evaluation
# REMEMBER TO DIS-COMMENT THE EXECUTION CODEs line 58

Cloning into '/content/human_eval'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 29 (delta 8), reused 4 (delta 4), pack-reused 8 (from 1)
Receiving objects: 100% (29/29), 54.20 KiB | 3.61 MiB/s, done.
Resolving deltas: 100% (9/9), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for evaluation: filename=evaluation-0.0.2-py3-none-any.whl size=2448 sha256=9c918ad97f7493d5784029dbeab536ed0439c39224731a2c54990654fab31edc
  Stored in directory: /root/.cache/pip/wheels/3d/f1/f4/6c643c9c66223d853b8525812f17ec2cf02bf665b0c16a043a
  Created wheel for python-gflags: filename=python_gflags-3.1.2-py3-none-any.whl size=57371 sha256=040eacb098a9573c53a2bf71a986d63aeed72651907481a5f9190dc60978cfaa
  Stored in directory: /root/.cache/pip/wheels/0d/7d/a2/e858f0632653

In [20]:
!pip install -e human_eval

Obtaining file:///content/human_eval
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=91141f58adc94033e7f36bc028a197c968f44a8022d60756e23c49a1b60c753b
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire
  Running setup.py develop for human-eval


In [32]:
def build_jsonl(mark_name):
    dict_file = ""
    import json
    files_length = len(os.listdir(f'/content/{mark_name}'))
    for idx in range(files_length):
        script = open(f'/content/{mark_name}/{idx}.py', 'r').read()
        try:
            script = extract_triple_quotes_content(script)[0]
        except:
            print(idx, end = ' ')
            script = script.split('```')[-1]
        dict_ = {"task_id": f"HumanEval/{idx}", "completion": script}
        dict_file += json.dumps(dict_)+ '\n'

    with open(f'/content/{mark_name}.jsonl', 'w') as f:
        f.write(dict_file)
build_jsonl('kgw')
print()
build_jsonl('sweet')
print()
build_jsonl('cc')
print()
build_jsonl('raw_ds_prompted_1.3b')


10 32 38 50 64 98 151 
130 
32 75 
64 129 

In [25]:
!evaluate_functional_correctness /content/sweet.jsonl --problem_file=/content/HumanEval.jsonl

Reading samples...
164it [00:00, 41130.46it/s]
Running test suites...
100% 164/164 [00:04<00:00, 33.72it/s]
Writing results to /content/sweet.jsonl_results.jsonl...
100% 164/164 [00:00<00:00, 43072.38it/s]
{'pass@1': 0.5487804878048781}


In [26]:
!evaluate_functional_correctness /content/kgw.jsonl --problem_file=/content/HumanEval.jsonl

Reading samples...
164it [00:00, 5528.00it/s]
Running test suites...
100% 164/164 [00:02<00:00, 67.82it/s]
Writing results to /content/kgw.jsonl_results.jsonl...
100% 164/164 [00:00<00:00, 69425.30it/s]
{'pass@1': 0.5426829268292683}


In [29]:
!evaluate_functional_correctness /content/cc.jsonl --problem_file=/content/HumanEval.jsonl

Reading samples...
164it [00:00, 7004.60it/s]
Running test suites...
100% 164/164 [00:03<00:00, 44.96it/s]
Writing results to /content/cc.jsonl_results.jsonl...
100% 164/164 [00:00<00:00, 67312.44it/s]
{'pass@1': 0.5121951219512195}


In [33]:
!evaluate_functional_correctness /content/raw_ds_prompted_1.3b.jsonl --problem_file=/content/HumanEval.jsonl

Reading samples...
164it [00:00, 11038.35it/s]
Running test suites...
100% 164/164 [00:02<00:00, 68.98it/s]
Writing results to /content/raw_ds_prompted_1.3b.jsonl_results.jsonl...
100% 164/164 [00:00<00:00, 75242.38it/s]
{'pass@1': 0.5304878048780488}
